In [ ]:
import sql
import pandas as pd
from functools import reduce
import datetime

In [ ]:
q='''select t1.ecosystem as ecosystem, advisories, advisory_with_CVEs, packages
from
(select ecosystem, count(distinct a.id) as advisories, count(distinct p.id) as packages
from advisory a
join package p on a.package_id = p.id
where type != 'Malicious Package'
group by ecosystem) as t1
join
(select ecosystem, count(distinct a.id) as advisory_with_CVEs
from advisoryCVE aC
join advisory a on aC.advisory_id = a.id
join package p on a.package_id = p.id
where type != 'Malicious Package'
group by ecosystem) as t2
on t1.ecosystem = t2.ecosystem
order by advisories desc;'''
df = pd.DataFrame(sql.execute(q))

In [ ]:
df['non-CVEs (%)'] = df['advisories'] - df['advisory_with_CVEs']
df['non-CVEs (%)'] = round (df['non-CVEs (%)'] / df['advisories'] * 100 , 2)
df['non-CVEs (%)'].median()

In [ ]:
df

In [ ]:
q='''select *
from advisory a
join package p on a.package_id = p.id
where a.type != 'Malicious Package'
and a.id not in
(select distinct advisory_id
    from advisoryCVE)'''
df = pd.DataFrame(sql.execute(q))
df['since_disclose'] = (datetime.date.today() - df['disclose_date']).dt.days
df['since_publish'] = (datetime.date.today() - df['publish_date']).dt.days
df['since_disclose'].median(), df['since_publish'].median()
p = df.groupby('ecosystem')[['since_publish']].median()
d = df.groupby('ecosystem')[['since_disclose']].median()
pd.merge(p,d,on='ecosystem')

In [ ]:
q = '''select * from
( select * from
    (select id, publish_date as snyk_publish, disclose_date, package_id
    from advisory
    where type != 'Malicious Package') as adv
    join
    (select advisory_id, min(publish_date) as cve_publish
    from advisoryCVE
    group by advisory_id) as cve
    on adv.id = cve.advisory_id
        where cve_publish > snyk_publish) as sub
join package p
    on p.id = sub.package_id;
'''
df = pd.DataFrame(sql.execute(q))
df['cve_delay'] = (df['cve_publish'] - df['snyk_publish']).dt.days
df['delay_from_disclose'] = (df['cve_publish'] - df['disclose_date']).dt.days
delay = df.groupby('ecosystem')[['cve_delay']].median()
count = df.groupby('ecosystem')[['id']].count()
disclose = df.groupby('ecosystem')[['delay_from_disclose']].median()
df=reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[count, delay, disclose])
df.columns = ['published by snyk before cve', 'median delay from snyk to cve', 'from disclose to cve']
df

In [ ]:
q='''select *
from advisory a
join package p on a.package_id = p.id
where type != 'Malicious Package' and ecosystem!='cocoapods';'''
df = pd.DataFrame(sql.execute(q))
df['delay'] = (df['publish_date'] - df['disclose_date']).dt.days
df = df.groupby('ecosystem')[['delay']].median()
df